## Question 1: Solving Stochastic Growth Model Numerically

### 1. Deriving the deterministic steady state analytically.

If productivity is non-random, we can drop the expectation operator:
$$\max_{\{c_t\}_{t=0}^\infty} \sum_{t=0}^{\infty} \beta^t \log(c_t).$$

Similarly, with $z_t = 0, \forall t$ we have the following simplified resource constraint:
$$c_t + k_{t+1} = k_t^\theta + (1-\delta)k_t \implies c_t = k_t^\theta + (1-\delta)k_t - k_{t+1}.$$

We can then write the Bellman equation where $k$ is the state variable. Substituting for $c$, we treat $k'$ as the choice variable and have:
$$V(k) = \max_{k'} \left\{ \log(k^\theta + (1-\delta)k - k') + \beta V(k') \right\}.$$

To find the steady state, we first derive the first-order condition.
$$\frac{\partial}{\partial k'}: \quad \beta V'(k') = \frac{1}{c}$$

Applying the envelope theorem to the state variable we obtain
$$V'(k) = \frac{1}{c} (\theta k^{\theta-1} + 1 - \delta).$$
By pushing the time one period forward, we can substitute and finally find the euler equation:
$$\frac{c'}{c} = \beta (1 - \delta + \theta (k')^{\theta-1}).$$

In the steady state, all variables are constant and thus also consumption growth is stable. By setting $c'/c = 1$, we then get:
$$1 = \beta (1 - \delta + \theta k^{\theta-1}).$$
Solving for $k$ we retrieve the steady state level of capital:
$$k^*_{ss} = \left[ \frac{\theta}{\frac{1}{\beta} - (1-\delta)} \right]^{\frac{1}{1-\theta}}.$$

If we impose the steady state to the (deterministic) resource constraint we know that $c^*_{ss} = (k^*_{ss})^\theta - \delta k^*_{ss}$, yielding

$$c^*_{ss} = \left[ \frac{\theta}{\frac{1}{\beta} - (1-\delta)} \right]^{\frac{\theta}{1-\theta}}-\delta \left[ \frac{\theta}{\frac{1}{\beta} - (1-\delta)} \right]^{\frac{1}{1-\theta}}$$
for the value of steady state consumption.

### 2. Numerically solving for the policy functions.

The following parameters are used:
- $\delta = 0.1$
- $\theta = 0.4$
- $\beta = 0.96$
- $q = 0.7$
- $\sigma = 0.1$

The transition matrix is thus

$$\begin{bmatrix} q & 1-q \\ 1-q & q \end{bmatrix} = \begin{bmatrix} 0.7 & 0.3 \\ 0.3 & 0.7 \end{bmatrix}$$

and the state space becomes $\{\sigma, -\sigma\} = \{0.1,-0.1\}.$ 

Given states $z, z' \in \{0.1,-0.1\}$, this (symmetric) markov chain describes the conditional probabilities of moving from the productivity state $z$ to the state $z'$. The diagonal entries ($q$) correspond to the probability of staying on the same productivity level from one period to the other, while the off-diagonal entries ($1-q$) correspond to the probability of moving from $z$ to $z'$ such that $z \neq z'$.